In [1]:
import pickle
import pandas as pd
import numpy as np
from pathlib import Path  
import os
import torch
from tqdm import tqdm

In [2]:
USE_ALL_FEATURES=False

model_name = 'enet_b2_8'
IMG_SIZE = 224

PATH='C:/Users/User/Downloads/face-emotion-recognition-main/models/affectnet_emotions/' + model_name + '.pt'

feature_extractor_model = torch.load(PATH)

MODEL2EMOTIW_FEATURES='C:/Users/User/Downloads/'+ model_name +'_cropped.pickle'

emotion_ind2label={0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happiness', 5: 'Neutral', 6: 'Sadness', 7: 'Surprise'}

In [3]:
classifier_weights=feature_extractor_model.classifier.weight.cpu().data.numpy()
classifier_bias=feature_extractor_model.classifier.bias.cpu().data.numpy()
print(classifier_weights.shape,classifier_weights)
print(classifier_bias.shape,classifier_bias)

(8, 1408) [[-0.08584318 -0.01135117  0.01391132 ...  0.05151795  0.04111349
  -0.03133564]
 [ 0.05045482  0.0124158   0.01706044 ...  0.00231916  0.03906726
   0.01346489]
 [-0.04731339  0.01440803 -0.02087102 ...  0.03216076 -0.00023583
   0.03391237]
 ...
 [-0.0733668  -0.02527069  0.01091044 ...  0.00450477 -0.02584679
   0.02445539]
 [ 0.00541873 -0.01534162 -0.02201885 ...  0.03147097  0.04480692
   0.04357051]
 [ 0.04042469 -0.01425645 -0.00016433 ... -0.03822443 -0.03345024
  -0.02694497]]
(8,) [-0.02906979  0.03788731  0.02125058  0.02583097 -0.04595341 -0.01233763
 -0.01429024 -0.02879   ]


In [4]:
def get_probab(features):
    x=np.dot(features,np.transpose(classifier_weights))+classifier_bias 
    #print(x)
    e_x = np.exp(x - np.max(x,axis=0))
    return e_x / e_x.sum(axis=1)[:,None]

In [5]:
with open(MODEL2EMOTIW_FEATURES, 'rb') as handle:
    filename2features=pickle.load(handle)
print(len(filename2features))

121


In [6]:
data_dir = r'C:\Users\User\Downloads\RealLifeDeceptionDetection.2016\Faces\mtcnn_cropped\Train'

## Enet 120 Samples

### Transitions Dataframe

In [ ]:
emotions = ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Anger', 'Disgust', 'Contempt']

transitions_head = ['video_name'] + [i+'-'+j for i in emotions for j in emotions] + ['target']
transitions_content = [] + [[] for i in range(len(transitions_head))] + []
transitions = dict(zip(transitions_head, transitions_content))

filenum = 0
for filename in os.listdir(data_dir): 
    for i in transitions:
        # print(transitions[i], '---', type(transitions[i]))
        if i != 'video_name' and i != 'target':
            transitions[i].append(0)
    transitions['video_name'].append(filename)

    frames_probabs=get_probab(filename2features[filename]) # получаем вероятности всех кадров видео
    frames_emotions = [emotion_ind2label[i.argmax()] for i in frames_probabs]

    # print(frames_emotions)

    for j in range(1, len(frames_emotions)-1):
        if frames_emotions[j] != frames_emotions[j-1]:
            transitions[frames_emotions[j-1] + '-' + frames_emotions[j]][filenum] += 1
            # print('1 -- ', frames_emotions[j-1] + '-' +  frames_emotions[j])
        elif frames_emotions[j] == frames_emotions[j-1]:
            if frames_emotions[j] != frames_emotions[j+1]:
                transitions[frames_emotions[j-1] + '-' + frames_emotions[j]][filenum] += 1
                # print('2 -- ', frames_emotions[j-1] + '-' +  frames_emotions[j])

    if transitions['video_name'][filenum][6:11] == 'truth':
        transitions['target'].append(1)
    else:
        transitions['target'].append(0)

    filenum += 1    

df = pd.DataFrame(transitions)

In [116]:
df

,video_name,Neutral-Neutral,Neutral-Happiness,Neutral-Sadness,Neutral-Surprise,Neutral-Fear,Neutral-Anger,Neutral-Disgust,Neutral-Contempt,Happiness-Neutral,...,Disgust-Contempt,Contempt-Neutral,Contempt-Happiness,Contempt-Sadness,Contempt-Surprise,Contempt-Fear,Contempt-Anger,Contempt-Disgust,Contempt-Contempt,target
0,trial_lie_001,16,0,2,2,0,11,10,1,0,...,8,4,2,0,0,0,2,7,11,0
1,trial_lie_002,35,6,2,11,0,1,16,2,11,...,1,2,1,0,0,0,0,1,4,0
2,trial_lie_003,5,0,1,0,2,1,1,0,0,...,0,0,0,0,0,1,0,0,1,0
3,trial_lie_004,9,1,0,2,1,0,3,2,2,...,1,2,1,0,0,0,0,1,4,0
4,trial_lie_005,40,8,3,11,6,0,16,1,10,...,0,1,0,0,1,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,trial_truth_056,2,0,1,2,0,1,0,0,0,...,3,1,1,5,1,1,8,5,9,1
117,trial_truth_057,12,0,0,0,0,4,18,2,0,...,3,2,0,0,0,0,0,3,3,1
118,trial_truth_058,4,0,1,0,0,0,1,3,0,...,13,2,2,0,1,0,5,14,18,1
119,trial_truth_059,18,0,0,2,0,15,12,0,0,...,0,0,0,0,0,0,1,0,1,1


In [117]:
filepath = Path(r'C:\Users\User\Desktop\Course_work_practice\transitions_dict_120.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 

### Emotions Dataframe

In [125]:
emotion_list = ['video_name', 'Anger', 'Contempt', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']

emotions_content = [] + [[] for i in range(len(emotion_list))]
emotions = dict(zip(emotion_list, emotions_content))

filenum = 0
for filename in os.listdir(data_dir): 
    for i in emotions:
        # print(transitions[i], '---', type(transitions[i]))
        if i != 'video_name':
            emotions[i].append(0)
    emotions['video_name'].append(filename)

    frames_probabs=get_probab(filename2features[filename]) # получаем вероятности всех кадров видео
    frames_emotions = [emotion_ind2label[i.argmax()] for i in frames_probabs]

    for j in range(len(frames_emotions)):
        emotions[frames_emotions[j]][filenum] += 1

    filenum += 1    
    

df = pd.DataFrame(emotions)

In [126]:
df.head()

,video_name,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,trial_lie_001,83,50,223,7,35,99,8,5
1,trial_lie_002,50,31,767,16,455,227,138,190
2,trial_lie_003,21,3,10,119,0,34,11,14
3,trial_lie_004,9,18,78,82,57,66,24,15
4,trial_lie_005,28,18,517,174,154,330,168,181


In [127]:
filepath = Path(r'C:\Users\User\Desktop\Course_work_practice\emotions_dict_120.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 

## Enet 240 Samples

### Transitions Dataframe

In [122]:
emotions = ['Neutral', 'Happiness', 'Sadness', 'Surprise', 'Fear', 'Anger', 'Disgust', 'Contempt']

transitions_head = ['video_name'] + [i+'-'+j for i in emotions for j in emotions] + ['target']
transitions_content = [] + [[] for i in range(len(transitions_head))] + []
transitions = dict(zip(transitions_head, transitions_content))

filenum = 0
divider = 2

for filename in os.listdir(data_dir): 
    for part in range(divider):
        for i in transitions:
            if i != 'video_name' and i != 'target':
                transitions[i].append(0)
        transitions['video_name'].append(filename + '_' + str(part))

        frames_probabs=get_probab(filename2features[filename]) # получаем вероятности всех кадров filename
        frames_emotions = [[emotion_ind2label[frames_probabs[i].argmax()] for i in range(len(frames_probabs) // 2)], 
        [emotion_ind2label[frames_probabs[i].argmax()] for i in range(len(frames_probabs) - 1, len(frames_probabs) // 2 - 1, -1)]]

        for j in range(1, len(frames_emotions[part]) - 1):
            if frames_emotions[part][j] != frames_emotions[part][j-1]:
                transitions[frames_emotions[part][j-1] + '-' + frames_emotions[part][j]][filenum] += 1
            elif frames_emotions[part][j] == frames_emotions[part][j-1]:
                if frames_emotions[part][j] != frames_emotions[part][j+1]:
                    transitions[frames_emotions[part][j-1] + '-' + frames_emotions[part][j]][filenum] += 1
            
        if transitions['video_name'][filenum][6:11] == 'truth':
            transitions['target'].append(1)
        else:
            transitions['target'].append(0)

        filenum += 1    

df = pd.DataFrame(transitions)

In [123]:
df

,video_name,Neutral-Neutral,Neutral-Happiness,Neutral-Sadness,Neutral-Surprise,Neutral-Fear,Neutral-Anger,Neutral-Disgust,Neutral-Contempt,Happiness-Neutral,...,Disgust-Contempt,Contempt-Neutral,Contempt-Happiness,Contempt-Sadness,Contempt-Surprise,Contempt-Fear,Contempt-Anger,Contempt-Disgust,Contempt-Contempt,target
0,trial_lie_001_0,3,0,0,1,0,1,3,0,0,...,1,1,2,0,0,0,0,1,4,0
1,trial_lie_001_1,12,0,2,0,0,8,7,3,0,...,6,1,1,0,0,0,2,7,7,0
2,trial_lie_002_0,21,4,1,6,0,1,9,2,4,...,0,2,1,0,0,0,0,0,3,0
3,trial_lie_002_1,14,7,1,1,0,0,6,0,2,...,1,0,0,0,0,0,0,1,1,0
4,trial_lie_003_0,5,0,1,0,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,trial_truth_058_1,1,0,0,0,0,0,1,0,0,...,10,1,1,0,2,0,1,9,10,1
238,trial_truth_059_0,8,0,0,2,0,6,5,0,0,...,0,0,0,0,0,0,0,0,0,1
239,trial_truth_059_1,10,0,0,0,0,8,8,0,0,...,0,0,0,0,0,0,1,0,1,1
240,trial_truth_060_0,6,0,1,1,5,2,0,0,0,...,0,2,0,1,1,0,0,0,4,1


In [124]:
filepath = Path(r'C:\Users\User\Desktop\Course_work_practice\transitions_dict_240.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 

### Emotions Dataframe

In [131]:
emotion_list = ['video_name', 'Anger', 'Contempt', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']

emotions_content = [] + [[] for i in range(len(emotion_list))]
emotions = dict(zip(emotion_list, emotions_content))

filenum = 0
for filename in os.listdir(data_dir): 
    for part in range(divider):
        for i in emotions:
            # print(transitions[i], '---', type(transitions[i]))
            if i != 'video_name':
                emotions[i].append(0)
        emotions['video_name'].append(filename + '_' + str(part))

        frames_probabs=get_probab(filename2features[filename]) # получаем вероятности всех кадров filename
        frames_emotions = [[emotion_ind2label[frames_probabs[i].argmax()] for i in range(len(frames_probabs) // 2)], 
        [emotion_ind2label[frames_probabs[i].argmax()] for i in range(len(frames_probabs) - 1, len(frames_probabs) // 2 - 1, -1)]]

        for j in range(len(frames_emotions[part])):
            emotions[frames_emotions[part][j]][filenum] += 1

        filenum += 1    
    

df = pd.DataFrame(emotions)

In [132]:
df

,video_name,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,trial_lie_001_0,40,18,136,7,31,17,4,2
1,trial_lie_001_1,43,32,87,0,4,82,4,3
2,trial_lie_002_0,18,28,349,14,228,121,60,119
3,trial_lie_002_1,32,3,418,2,227,106,78,71
4,trial_lie_003_0,21,0,9,34,0,34,6,2
...,...,...,...,...,...,...,...,...,...
237,trial_truth_058_1,18,41,214,12,4,2,0,24
238,trial_truth_059_0,75,0,255,3,1,40,3,28
239,trial_truth_059_1,65,5,271,2,0,60,0,2
240,trial_truth_060_0,32,13,15,128,0,45,24,13


In [133]:
filepath = Path(r'C:\Users\User\Desktop\Course_work_practice\emotions_dict_240.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath) 